<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/ucsd-logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/ndp-logo.png" alt="NDP Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/open-topo.png" alt="OpenTopo Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">OpenTopo Demo</h1>

### Import Libraries

In [ ]:
from pelicanfs.core import PelicanFileSystem, PelicanMap
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import plotting_extent
import rioxarray
import fsspec
from pathlib import Path

## Inyo Domes, CA lidar 2016

This dataset, located northwest of Mammoth Lakes, CA, was collected as an NCALM Seed grant for PI Abigail Martens, University of California, Bakersfield, to for structural analysis of obsidian lava surfaces. PI: California State University, Bakersfield, Department of Geological Sciences 9001 Stockdale Hwy, Bakersfield, CA 93311Publications associated with this dataset can be found at NCALM's Data Tracking Center.

### Load Data

In [ ]:
tif_path = "./inyo-domes-ca-lidar-20163/319_4171_begd.tif"

with rasterio.open(tif_path) as ds:
    dtm = ds.read(1).astype(np.float32)
    nodata = ds.nodata
    extent = plotting_extent(ds)
    crs = ds.crs
    transform = ds.transform

print("shape (rows, cols):", dtm.shape)
print("dtype:", dtm.dtype)
print("nodata:", nodata)
print("crs:", crs)
print("transform:", transform)
print("min/max (raw):", np.nanmin(dtm), np.nanmax(dtm))

### DTM Visualization

In [ ]:
if nodata is not None:
    dtm = np.where(dtm == nodata, np.nan, dtm)

print("min/max (masked):", np.nanmin(dtm), np.nanmax(dtm))

# Robust stretch for visualization (2–98%)
lo, hi = np.nanpercentile(dtm, [2, 98])
img = np.clip((dtm - lo) / (hi - lo + 1e-12), 0, 1)

plt.figure(figsize=(10, 8))
plt.imshow(img, extent=extent)
plt.title("DTM (2–98% stretch)")
plt.xlabel("x")
plt.ylabel("y")
plt.colorbar(label="stretched value")
plt.show()

## Boulder Creek Critical Zone Observatory - May 2010 

Data were collected in collaboration between the National Center for Airborne Laser Mapping (NCALM) project and the Boulder Creek Critical Zone Observatory (CZO), both funded by the National Science Foundation (NSF). The dataset contains 1 m Digital Surface Models (first-stop), Digital Terrain Models (bare-earth), and 10 points/m2 LAS-formated point cloud tiles. The DSMs and DTMs are available in GeoTIFF format with associated shaded relief models.

Original dataset can be found in [this site](https://nationaldataplatform.org/dataset/boulder-creek-critical-zone-observatory-may-2010-snow-on-lidar-survey-geotiff).

### Load Data

Original files are stored within the [Open Science Data Federation (OSDF)](https://osg-htc.org/services/osdf) We'll fetch the data using the [Pelican File System](https://docs.pelicanplatform.org/about-pelican).

There are 3 records of GeoTIFF Files:
- BCZO_SnowOn09
- BCZO_SnowOn05
- BCZO_SnowOn2021
  
Let's look at 09:

In [ ]:
# Defining the federation and the namespace
federation = "osg-htc.org"
namespace = "/ndp/burnpro3d/BCZO_SnowOn09"
pelfs = PelicanFileSystem(f"pelican://{federation}")

In [ ]:
# Printing objects in the directory
for obj in pelfs.ls(namespace):
    print(obj)

There is another subdirectory we can explore `/ndp/burnpro3d/BCZO_SnowOn09/BCZO_SnowOn09_be`

In [ ]:
# We update the namespace
namespace = "/ndp/burnpro3d/BCZO_SnowOn09/BCZO_SnowOn09_be"
for obj in pelfs.ls(namespace):
    print(obj)

`BcCZO_DTM_20100509_SnOn.tif` is the Digital Terrain Model we will use for exploration.

In [ ]:
# Use file path
fpath = "osdf:///" + "/ndp/burnpro3d/BCZO_SnowOn09/BCZO_SnowOn09_be/BcCZO_DTM_20100509_SnOn.tif"

In [ ]:
with fsspec.open(str(fpath), "rb") as fi:
    print(fi)
    da = rioxarray.open_rasterio(fi, masked=True)

print(da)
print("CRS:", da.rio.crs)
print("Bounds:", da.rio.bounds())
print("Resolution:", da.rio.resolution())

### Elevation Map

In [ ]:
da.plot(cmap="terrain", figsize=(10,8))
plt.title("DTM Elevation")
plt.show()

## DTM 3D Surface

In [ ]:
da_masked = da.where(np.isfinite(da))
da_masked = da_masked.where((da_masked > 1500) & (da_masked < 5000))

# Downsample
da_small = da_masked.coarsen(x=10, y=10, boundary="trim").mean()

Z = da_small[0].values
Z = np.where(np.isfinite(Z), Z, np.nan)

xs = da_small.x.values
ys = da_small.y.values
X, Y = np.meshgrid(xs, ys)

# Define fig
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection="3d")

# Plot surface
surf = ax.plot_surface(
    X, Y, Z,
    cmap="terrain",
    linewidth=0,
    antialiased=False,
    rstride=1,
    cstride=1
)

fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Elevation (m)")

ax.set_title("DTM 3D Surface (sample)")
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
ax.set_zlabel("Elevation (m)")

ax.view_init(elev=60, azim=-120)

plt.show()

### Hillshade + Slope Map

In [ ]:
# Get the 2D DEM (drop the band dimension)
dem = da.sel(band=1)

Z = dem.values

# Extract cell size from coordinates
dy = float(abs(dem.y[1] - dem.y[0]))
dx = float(abs(dem.x[1] - dem.x[0]))

# Compute gradients
dz_dy, dz_dx = np.gradient(Z, dy, dx)

# Slope in degrees
slope_rad = np.arctan(np.hypot(dz_dx, dz_dy))
slope_deg = np.degrees(slope_rad)

# Hillshade
azimuth = 315  # NW light
altitude = 45  # mid-height sun

az = np.deg2rad(azimuth)
alt = np.deg2rad(altitude)

slope = np.pi / 2 - slope_rad
aspect = np.arctan2(-dz_dx, dz_dy)

hillshade = (
    np.sin(alt) * np.sin(slope) +
    np.cos(alt) * np.cos(slope) * np.cos(az - aspect)
)

# Normalize hillshade 0–1
hillshade = (hillshade - hillshade.min()) / (hillshade.max() - hillshade.min())

# Plot: hillshade + slope
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(hillshade, cmap="gray")
axes[0].set_title("Hillshade (Shaded Relief)")
axes[0].axis("off")

im = axes[1].imshow(slope_deg, cmap="viridis")
axes[1].set_title("Slope (degrees)")
axes[1].axis("off")
fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04, label="Slope (°)")

plt.tight_layout()
plt.show()